# Importe De Librerías

In [253]:
import pandas as pd
import numpy as np
import xgboost as xgb
import sklearn

from sklearn.preprocessing import scale, MinMaxScaler
from sklearn.model_selection import StratifiedKFold, KFold,RandomizedSearchCV
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import make_scorer
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import mean_squared_error



# Regresión

Lectura de archivos

In [254]:
ds_regresion = pd.read_csv("properati_argentina_depurado_train.csv")
test = pd.read_csv("properati_argentina_depurado_test.csv")

XGBoost

In [255]:
ds_prop_XGBoost_train = ds_regresion.copy()
ds_prop_XGBoost_test = test.copy()

ds_prop_XGBoost_train = ds_prop_XGBoost_train.dropna()
ds_prop_XGBoost_test = ds_prop_XGBoost_test.dropna()

Realizamos la ingeniería de caracteristicas para cada dataset

In [256]:
#Eliminamos las features sin importancia

variables_eliminadas = ["start_date", "end_date", "created_on", "property_title", "place_l2", "property_currency", "operation", "geometry", "cluster"]
ds_prop_XGBoost_train = ds_prop_XGBoost_train.drop(variables_eliminadas, axis='columns', inplace=False)

variables_eliminadas = ["start_date", "end_date", "created_on", "property_title", "place_l2", "property_currency", "operation"]
ds_prop_XGBoost_test = ds_prop_XGBoost_test.drop(variables_eliminadas, axis='columns', inplace=False)


In [257]:
#One Hot Encoding para variables categoricas

variables_reemplazadas = ["property_type", "place_l3"]
ds_prop_XGBoost_train = pd.get_dummies(ds_prop_XGBoost_train, columns=variables_reemplazadas, drop_first=True)
ds_prop_XGBoost_test = pd.get_dummies(ds_prop_XGBoost_test, columns=variables_reemplazadas, drop_first=True)

#Escalamos las variables para que no tengan mayor peso
scaler = MinMaxScaler()

ds_prop_XGBoost_train["property_surface_total"] = scaler.fit_transform(ds_prop_XGBoost_train["property_surface_total"].to_frame())
ds_prop_XGBoost_train["longitud"] = scaler.fit_transform(ds_prop_XGBoost_train["longitud"].to_frame())
ds_prop_XGBoost_train["latitud"] = scaler.fit_transform(ds_prop_XGBoost_train["latitud"].to_frame())

ds_prop_XGBoost_test["property_surface_total"] = scaler.fit_transform(ds_prop_XGBoost_test["property_surface_total"].to_frame())
ds_prop_XGBoost_test["longitud"] = scaler.fit_transform(ds_prop_XGBoost_test["longitud"].to_frame())
ds_prop_XGBoost_test["latitud"] = scaler.fit_transform(ds_prop_XGBoost_test["latitud"].to_frame())




In [258]:
#Hacemos division Train-Test
features = ['property_rooms','property_bedrooms','property_surface_total','property_surface_covered','latitud','longitud', 'property_type_PH', 'property_type_Departamento']
target = ["property_price"]

x_train = ds_prop_XGBoost_train[features]
x_test = ds_prop_XGBoost_test[features]

y_train = ds_prop_XGBoost_train[target]
y_test = ds_prop_XGBoost_test[target]

Buscamos los mejores hiperparametros usando Random CV. Usamos Random CV para hallar los hiperparametros que optimizen el MSE, ya que es la medida que nos interesa a al hora de hacer regresion. Queremos calcular el error medio de las predicciones

In [259]:
##KFOLD CV Random Search para buscar el mejor arbol (los mejores atributos, hiperparametros,etc)

#Cantidad de combinaciones que quiero porbar
n=5

#Conjunto de parámetros que quiero usar
params_grid = {'criterion':['gini','entropy'],
               #'min_samples_leaf':list(range(1,10)),
               #'min_samples_split': list(range(2,20)),
               'ccp_alpha':np.linspace(0,0.05,n), 
               'max_depth':list(range(0, 15)),
               'random_state':list(range(0,6))}
                
#Cantidad de splits para el Cross Validation
folds=20

#Kfold estratificado
#kfoldcv = StratifiedKFold(n_splits=folds)

#Regresor
xgb_model_rd_search = xgb.XGBRegressor()

#Metrica que quiero optimizar MSE
scorer_fn = make_scorer(sklearn.metrics.mean_squared_error)

#Random Search Cross Validation
randomcv = RandomizedSearchCV(estimator=xgb_model_rd_search,
                              param_distributions = params_grid,
                              scoring=scorer_fn,
                              n_iter=n, cv=folds) 

#GridSearch CV
#gridcv = GridSearchCV(estimator=xgb_model_rd_search,
#                      param_grid=params_grid,
#                      scoring=scorer_fn,
#                      return_train_score='True') 

#Busco los hiperparamtros que optimizan F1 Score
randomcv.fit(x_train,y_train)

[06:40:09] WARNING: C:/Users/administrator/workspace/xgboost-win64_release_1.6.0/src/learner.cc:627: 
Parameters: { "ccp_alpha", "criterion" } might not be used.

  This could be a false alarm, with some parameters getting used by language bindings but
  then being mistakenly passed down to XGBoost core, or some parameter actually being used
  but getting flagged wrongly here. Please open an issue if you find any such cases.


[06:40:10] WARNING: C:/Users/administrator/workspace/xgboost-win64_release_1.6.0/src/learner.cc:627: 
Parameters: { "ccp_alpha", "criterion" } might not be used.

  This could be a false alarm, with some parameters getting used by language bindings but
  then being mistakenly passed down to XGBoost core, or some parameter actually being used
  but getting flagged wrongly here. Please open an issue if you find any such cases.


[06:40:11] WARNING: C:/Users/administrator/workspace/xgboost-win64_release_1.6.0/src/learner.cc:627: 
Parameters: { "ccp_alpha", "criterio

C:\Users\Carreño\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\sklearn\model_selection\_validation.py:378: FitFailedWarning: 
20 fits failed out of a total of 100.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
19 fits failed with the following error:
Traceback (most recent call last):
  File "C:\Users\Carreño\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\sklearn\model_selection\_validation.py", line 686, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "C:\Users\Carreño\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-pa

[06:43:05] WARNING: C:/Users/administrator/workspace/xgboost-win64_release_1.6.0/src/learner.cc:627: 
Parameters: { "ccp_alpha", "criterion" } might not be used.

  This could be a false alarm, with some parameters getting used by language bindings but
  then being mistakenly passed down to XGBoost core, or some parameter actually being used
  but getting flagged wrongly here. Please open an issue if you find any such cases.




RandomizedSearchCV(cv=20,
                   estimator=XGBRegressor(base_score=None, booster=None,
                                          callbacks=None,
                                          colsample_bylevel=None,
                                          colsample_bynode=None,
                                          colsample_bytree=None,
                                          early_stopping_rounds=None,
                                          enable_categorical=False,
                                          eval_metric=None, gamma=None,
                                          gpu_id=None, grow_policy=None,
                                          importance_type=None,
                                          interaction_constraints=None,
                                          learning_rate=None, max_bin=None,...
                                          n_estimators=100, n_jobs=None,
                                          num_parallel_tree=None,
                                          predictor=None, random_state=None,
                                          reg_alpha=None, reg_lambda=None, ...),
                   n_iter=5,
                   param_distributions={'ccp_alpha': array([0.    , 0.0125, 0.025 , 0.0375, 0.05  ]),
                                        'criterion': ['gini', 'entropy'],
                                        'max_depth': [0, 1, 2, 3, 4, 5, 6, 7, 8,
                                                      9, 10, 11, 12, 13, 14],
                                        'random_state': [0, 1, 2, 3, 4, 5]},
                   scoring=make_scorer(mean_squared_error))

In [260]:
#Mejores hiperparametros
print(randomcv.best_params_)

#Mejor métrica
mse = randomcv.best_score_
print("RMSE en datos de entrnamiento: " + str(np.sqrt(mse)))

{'random_state': 0, 'max_depth': 3, 'criterion': 'entropy', 'ccp_alpha': 0.0}
RMSE en datos de entrnamiento: 101528.17304703005


Entrenamos el modelo

In [261]:
xgb_model = xgb.XGBRegressor().set_params(**randomcv.best_params_)
xgb_model.fit(x_train, y_train)

[06:43:06] WARNING: C:/Users/administrator/workspace/xgboost-win64_release_1.6.0/src/learner.cc:627: 
Parameters: { "ccp_alpha", "criterion" } might not be used.

  This could be a false alarm, with some parameters getting used by language bindings but
  then being mistakenly passed down to XGBoost core, or some parameter actually being used
  but getting flagged wrongly here. Please open an issue if you find any such cases.




XGBRegressor(base_score=0.5, booster='gbtree', callbacks=None, ccp_alpha=0.0,
             colsample_bylevel=1, colsample_bynode=1, colsample_bytree=1,
             criterion='entropy', early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, gamma=0, gpu_id=-1,
             grow_policy='depthwise', importance_type=None,
             interaction_constraints='', learning_rate=0.300000012, max_bin=256,
             max_cat_to_onehot=4, max_delta_step=0, max_depth=3, max_leaves=0,
             min_child_weight=1, missing=nan, monotone_constraints='()',
             n_estimators=100, n_jobs=0, num_parallel_tree=1, predictor='auto',
             random_state=0, ...)

Medimos que tan precisas fueron las predicciones usando RMSE. La ventaja de usar RMSE en vez de MSE es que el valor de RMSE está en las mismas unidades que la variable target. De esta forma es más fácil dimensionar que tan preciso es el modelo creado

In [262]:
from sklearn.metrics import mean_squared_error


y_pred = xgb_model.predict(x_test)
#MSE 121489.55

rmse = mean_squared_error(y_test, y_pred, squared=False)
#print("MSE: %.2f" % (mse**(1/2.0)))
print("RMSE en datos de test: " + str(rmse))

RMSE en datos de test: 184088.99927721854


Vemos que la performance de entrenamiento y la del conjunto de evaluación es similar. Esto significa que no hay overfitting, es decir, el modelo no se aprende de 'memoria' los datos de entrenamiento y no pierde precision al realizar las predicciones en el conjunto de evaluación. 

In [263]:
#Atributos considerados y su importancia

features = ["latitud", "longitud", "property_rooms", "propperty_bedrooms", "property_surface_total", "property_surface_covered", "property_type_PH", "property_type_Departamento"]
feat_imps = xgb_model.feature_importances_

for feat_imp,feat in sorted(zip(feat_imps,features)):
  if feat_imp>0:
    print('{}: {}'.format(feat,feat_imp))


latitud: 0.014410820789635181
longitud: 0.014815220609307289
property_surface_covered: 0.038302745670080185
property_surface_total: 0.055447451770305634
property_rooms: 0.08408662676811218
property_type_PH: 0.08468924462795258
propperty_bedrooms: 0.33298277854919434
property_type_Departamento: 0.37526506185531616
